## 단위 테스트는 무엇이고, 기능 테스트와 어떤 차이가 있을까?

기본적인 차이는 기능 테스트는 사용자 관점에서 애플리케이션 외부를 테스트하는 것이고, 단위 테스트는 프로그래머 관점에서 그 내부를 테스트한다는 것이다.

1. 기능 테스트를 작성해서 사용자 관점의 새로운 기능성을 정의하는 것부터 시작한다.
2. 기능 테스트가 실패하고 나면 어떻게 코드를 작성해야 테스트를 통과할지를 생각해보도록 한다. 이 시점에서 하나 또는 그 이상의 단위 테스트를 이용해서 어떻게 코드가 동작해야 하는지 정의한다.
3. 단위 테스트가 실패하고 나면 단위 테스트를 통과할 수 있을 정도의 최소한의 코드만 작성한다. 기능 테스트가 완전해질 때까지 2와 3을 반복해야할 수 있다.
4. 기능 테스트를 재실행해서 통과하는지 또는 제대로 동작하는지 확인한다. 이 과정에서 새로운 단위 테스트를 작성해야 할 수도 있다.

이 과정을 잘 보면, 기능 테스트는 상위 레벨의 개발을 주도하고 단위 테스트는 하위 레벨을 주도한다는 것을 알 수 있다

기능 테스트와 단위 테스트가 전혀 다른 목적을 가지고 있어서 서로 다른 결과를 초래할 수 있기 때문에 꼭 필요한 과정이라 할 수 있다.

## Django에서의 단위 테스트

TDD 주기는 실패 테스트를 작성한 후 테스트를 통과할 수 있는 코드를 작성하는 과정이라는 것을 보았다. 단위 테스트의 경우에는 그것이 어떠한 형태든 자동화된 테스트 실행자에 의해 실행된다는 것을 알아툴 필요가 있다. functional_tests.py는 직접 실행했지만 고의적인 실패 테스트를 만들어서 이를 확인해보자.

In [1]:
from django.test import TestCase

# Create your tests here.
class SmokeTest(TestCase):
	def test_bad_maths(self):
		self.assertEqual(1 + 1, 3)

이 미스터리한 테스트 실행자를 호출해보자. 이전과 같이 manage.py 명령을 이용하면 된다.

python3 manage.py test

## Django의 MVC, URL, 뷰 함수

Django는 대체로 모델, 뷰, 컨트롤러라는 고전적인 패턴을 따른다. "대체로"라는 표현에 주목하자. 왜냐하면 분명한 모델은 있지만, 뷰는 컨트롤러와 비슷한 역할을 하기 때문이다. 뷰는 보이는 부분을  담당하는 템플릿이라는 개념에 주안을 두고 있다.

Django의 처리 흐름은 다음과 같다.

1. 특정 URL에 대한 HTTP "요청"을 받는다.
2. Django는 특정 규칙을 이용해서 해당 요청에 어떤 뷰 함수를 실행할지 결정한다.
3. 이 뷰 기능이 요청을 처리해서 HTTP "응답"으로 반환한다.

따라서 우리가 테스트해야할 것은 두 가지다.

* URL의 사이트 루트("/")를 해석해서 특정 뷰 기능에 매칭시킬 수 있는가?
* 이 뷰 기능이 특정 HTML을 반환하게 해서 기능 테스트를 통과할 수 있는가?

In [4]:
# from django.core.rlresolvers import resolve
from django.urls import reverse
from lists.views import home_page # 1

class HomePageTest(TestCase):
	def test_root_url_resolves_to_home_page_view(self):
		found = resolve('/') # 2
		self.assertEqual(found.func, home_page) # 3

2, 3 resolve는 Django가 내부적으로 사용하는 함수로, URL을 해석해서 일치하는 뷰 함수를 찾는다. 여기서는 "/"가 호출될 때 resolve를 실행해서 home_page라는 함수를 호출한다.

1 home_page는 어떤 함수일까? 곧 작성하게 될 뷰 함수로 HTML을 반환한다. import 처리 부분을 보면 알 수 있지만 이 함수를 lists/views.py 파일에 저장할 계획이다.

## 마침내 실질적인 애플리케이션 코드를 작성한다

TDD는 인내를 요하는 작업으로 점진적으로 진행되며 개선 속도도 느리다. 특히 지금은 배우는 과정으로 이제 막 걸음마를 뗴었기 때문에, 한 줄의 코드만 수정해 갈 것이다.실패 테스트를 해결하기 위한 최소한의 수정만 하도록 한다.

lists.view에서 home_page를 임포트할 수 없는 상태다. 다음과 같이 이 문제를 해결해보자.

In [5]:
from django.shortcuts import render

home_page = None

## urls.py

Django는 urls.py라는 파일을 이용해서 어떻게 URL을 뷰 함수에 맵핑할지 정의한다. superlists/superlists 폴더에 전체 사이트를 대상으로 하는 메인 urls.py가 있다. 파일 내용을 살펴보자.

## 뷰를 위한 단위 테스트

HTML 형식의 실제 응답을 반환하는 함수를 작성해야 한다. lists/tests.py를 열어서 다음과 같이 새로운 테스트를 추가한다.

In [7]:
from django.urls import reverse
from lists.views import home_page # 1
from django.http import HttpRequest

class HomePageTest(TestCase):
	def test_root_url_resolves_to_home_page_view(self):
		found = reverse('/') # 2
		self.assertEqual(found.func, home_page) # 3


	def test_home_page_returns_correct_html(self):
		request = HttpRequest() # 1
		response = home_page(request) # 2
		self.assertTrue(response.content.startswith(b'<html>')) # 3
		self.assertIn(b'<title>To-Do lists</title>', response.content) # 4
		self.assertTrue(response.content.endswith(b'</html>')) # 5

1. HttpRequest 객체를 생성해서 사용자가 어떤 요청을 브라우저에 보내는지 확인한다.
2. 이것은 home_page 뷰에 전달해서 응답을 취득한다. 이 객체는 HttpResponse라는 클래스의 인스턴스다. 응답 내용이 특정 속성을 가지고 있는지 확인한다.
3. 응답 내용이 <html>로 시작하고 끝나는지 확인한다. response.content는 byte형 데이터로, 파이썬 문자열이 아니다. 따라서 b'' 구문을 사용해서 비교한다.
    
### 단위 테스트 - 코드 주기

1. 터미널에서 단위 테스트를 실행해서 어떻게 실패하는지 확인한다.
2. 편집기상에서 현재 실패 테스트를 수정하기 위한 최소한의 코드를 변경한다.

코드 품질을 높이고 싶다면 코드 변경을 최소화해야 한다. 또한 이렇게 최소화한 코드는 하나하나 테스트에 의해 검증돼야 한다. 매우 고된 작업이라고 생각될 수 있지만, 한번 익숙해지기 시작하면 속도는 빨라진다. 따라서 아무리 자신 있는 부분이라도 작은 단위로 나누어 코드를 변경하도록 한다.

얼마나 빨리 이 주기를 따라갈 수 있는지 보도록 하자.

* 최소한의 코드 변경

In [8]:
def home_page(request):
    pass

* 테스트

In [10]:
self.assertTrue(response.content.startswith(b'<html>'))

* 코드: 가정한 대로 django.http.httpResponse를 사용한다.

* 다시 테스트

In [12]:
self.assertTrue(response.content.startswith(b'<html>'))

* 다시 코드

In [14]:
def home_page():
    return httpResponse('<html><title>To-Do lists</title>')

* 테스트    

In [16]:
self.assertTrue(response.content.startswith(b'<html>'))

* 코드

In [18]:
def home_page():
    return httpResponse('<html><title>To-Do lists</title>')

최소한의 주기로 코드를 테스트를 반복하면서 진행하는 것이다.